<a href="https://colab.research.google.com/github/kazuki-kimura-prime/EZ_ARIMA/blob/main/EZ_ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EZ_ARIMA ： ARIMAモデルによる年間予測支援ツール

# EZ_ARIMAとは

## 📌 概要

**EZ_ARIMA** は、Pythonベースで構築された季節性ARIMA（SARIMA）モデルによる時系列データの年間予測支援ツールです。誰でも簡単にARIMAモデルを使用して中長期的な変動予測を行えるようにすることを目的として開発されました。

専門知識がないユーザーでも、時系列データの入力さえ行えば、12か月先までの予測とその評価を自動で出力します。


## 🔍 主な機能

- 月次データに対応した **SARIMAモデル** を使用した予測
- **自動パラメータ推定** によるモデル構築（`statsmodels` ライブラリを使用）
- 学習データと検証データの自動分割
- **MAE / RMSE / MAPE / R²** による精度評価
- **残差分析**（ACFプロット、Q-Qプロット）によるモデル妥当性確認
- **95%信頼区間**付き予測グラフの自動描画


## 🧪 活用事例

公開準備中


## 📊 対応データ


* 形式: CSV
* 頻度: 月次データ（毎月1回の観測データ）
* 長さ: 推奨5年以上（最低でも3年以上）




## 📈 出力する結果
* 実測値と予測値の重ね合わせグラフ
* 信頼区間付き将来予測グラフ
* 残差のヒストグラム、ACFプロット、Q-Qプロット
* 評価指標（MAE、RMSE、MAPE、R²）

## ⚠️ 注意点
* ARIMAモデルは時系列データに基づく予測を行います。突発的な異常や構造的変化がある場合、精度が低下する可能性があります。
* モデルの結果はあくまで補助的判断材料であり、**最終的な意思決定は利用者の責任で行ってください**。

# EZ_ARIMA

## 使い方

右に[　]があるまとまりを「セル」といいます。セルの前にある説明を読んで上から順に実行していってください。
それぞれのセルの[　]にカーソルを合わせると▶️が表示されます。クリックするとそのセルを実行できます。実行すると⏹️になりますので、しばらくお待ちください。[　]の隣に☑️が表示されれば正常に終了しています。❗️マークの時は実行に失敗しています。もう一度実行してください。

## 環境構築セクション

Google Driveに接続します。「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」と表示されるので、ご自身のアカウントで接続してください。

```
Mounted at /content/drive
```
と表示されれば成功です。


In [ ]:
# Google Driveに接続
from google.colab import drive
drive.mount('/content/drive')

次に、必要なライブラリをインストールします。「セッションを再起動する」と表示されますが正常です。

```
RESTERT SESSION を押してください
```
と表示されますので、そのボタンをクリックすれば終了です。


In [ ]:
!pip uninstall -y pandas matplotlib statsmodels pmdarima
!pip install pandas==2.2.2 matplotlib==3.8.0 statsmodels==0.14.0 pmdarima==2.0.3
print(f"セッションを再起動してください。（RESTERT SESSION を押してください）")

インストールされたバージョンを確認します。

```
pandas: 2.2.2
matplotlib: 3.8.0
statsmodels: 0.14.0
pmdarima: 2.0.3
```
と表示されれば終了です。もし、異なる場合は一つ上載せるからやり直してください。


In [ ]:
import pandas
import matplotlib
import statsmodels
import pmdarima

print("pandas:", pandas.__version__)
print("matplotlib:", matplotlib.__version__)
print("statsmodels:", statsmodels.__version__)
print("pmdarima:", pmdarima.__version__)

if pandas.__version__ != '2.2.2':
    print(f"pandasのバージョンが2.2.2ではありません。現在のバージョン: {pandas.__version__}")
if matplotlib.__version__ != '3.8.0':
    print(f"matplotlibのバージョンが3.8.0ではありません。現在のバージョン: {matplotlib.__version__}")
if statsmodels.__version__ != '0.14.0':
    print(f"statsmodelsのバージョンが0.14.0ではありません。現在のバージョン: {statsmodels.__version__}")
if pmdarima.__version__ != '2.0.3':
    print(f"pmdarimaのバージョンが2.0.3ではありません。現在のバージョン: {pmdarima.__version__}")

## データ作成セクション

CSV形式のデータ様式を出力します。わかるところにダウンロードしてください。その後、このファイルの中のデータを解析したいデータに書き換えてください。

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# ダミーデータを生成する
start_date = datetime(2020, 4, 1)  # 開始日
periods = 12 * 5  # 12か月 × 5年分のデータ
frequency = 'M'  # 月次データ

# 日付データの生成
date_range = pd.date_range(start=start_date, periods=periods, freq=frequency)

# SARIMA向けの値を生成
trend = np.linspace(50, 100, periods)  # トレンド
seasonality = 10 * np.sin(2 * np.pi * np.arange(periods) / 12)  # 季節性
noise = np.random.normal(0, 5, periods)  # ランダムノイズ
values = trend + seasonality + noise  # トレンド + 季節性 + ノイズ

# DataFrameを作成
data = pd.DataFrame({
    'date': date_range,
    'value': values
})

# CSVファイルとして保存
data.to_csv('data.csv', index=False)

# Google Colabからローカルにダウンロード
from google.colab import files
files.download('data.csv')

# 確認用にデータの先頭5行を表示
print(data.head())
print(f"様式となるCSVファイルを作成しました。")

## 解析セクション

最後の12ヶ月とそれ以外のデータに分けて解析を行います。最後の12ヶ月と解析結果を比較してモデルの精度をユーザーが確認する必要があります。

```
CSVファイルをアップロードしてください
```
と表示されますので、作成したCSVファイルを読み込んでください。その後、自動的に解析が実行されますので解析結果に問題がないか確認してください。結果の見方はこのセルのに記載しています


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pmdarima as pm
import numpy as np
import statsmodels.api as sm
import seaborn as sns
from pandas.tseries.offsets import MonthEnd
from google.colab import files
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# ======== ファイルアップロード ========
print("📂 CSVファイルをアップロードしてください")
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# ======== データ読み込み & 月末に補正 ========
df = pd.read_csv(filename)

# 必須カラム確認
if 'date' not in df.columns or 'value' not in df.columns:
    raise ValueError("CSVには 'date' 列と 'value' 列が必要です。")

# 日付変換 + 月末に補正
df['date'] = pd.to_datetime(df['date']) + MonthEnd(0)

# 時系列データとしてインデックス設定
df = df.sort_values('date')
df.set_index('date', inplace=True)

# ======== 時系列データの抽出 ========
ts = df['value']

# ======== 学習・テスト分割（最後の12ヶ月をテストに） ========
train = ts.iloc[:-12]
test = ts.iloc[-12:]
forecast_index = test.index

# ======== SARIMAモデルの自動構築と学習 ========
model = pm.auto_arima(
    train,
    seasonal=True,
    m=12,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)
print("\n✅ モデル概要:")
print(model.summary())

# ======== 予測の実行と信頼区間取得 ========
n_periods = 12
forecast_values, conf_int = model.predict(n_periods=n_periods, return_conf_int=True)

# Series に変換し、インデックスを月末に統一
forecast = pd.Series(forecast_values, index=forecast_index)
lower_ci = pd.Series(conf_int[:, 0], index=forecast_index)
upper_ci = pd.Series(conf_int[:, 1], index=forecast_index)

# ======== 比較用DataFrameの作成 ========
result_df = pd.DataFrame({
    'Actual': test,
    'Forecast': forecast,
    'Lower CI': lower_ci,
    'Upper CI': upper_ci
})
result_df['Error'] = result_df['Actual'] - result_df['Forecast']
result_df['Error (%)'] = (result_df['Error'] / result_df['Actual']) * 100

# ======== グラフ①：全体表示（Train + Test + Forecast） ========
plt.figure(figsize=(14, 6))
plt.plot(train.index, train, label='Train', color='blue', marker='s')
plt.plot(test.index, test, label='Actual (Test)', color='green', marker='s')
plt.plot(forecast.index, forecast, label='Forecast', color='red', linestyle='--', marker='o')
plt.fill_between(forecast.index, lower_ci, upper_ci,
                 color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title('SARIMA Forecast vs Actual with Train Overlay (Month End)')
plt.xlabel('Date (Month End)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ======== グラフ②：ズーム表示（12ヶ月） ========
plt.figure(figsize=(12, 5))
plt.plot(result_df.index, result_df['Actual'], label='Actual', color='green', marker='s')
plt.plot(result_df.index, result_df['Forecast'], label='Forecast', color='red', linestyle='--', marker='o')
plt.fill_between(result_df.index,
                 result_df['Lower CI'],
                 result_df['Upper CI'],
                 color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title('Forecast vs Actual (Last 12 Months, Month End)')
plt.xlabel('Date (Month End)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# ======== 表の出力 ========
print("\n📊 予測 vs 実データ（最後の12ヶ月）")
print(result_df.round(2))


# ======== Evaluation Metrics ========
mae = mean_absolute_error(result_df['Actual'], result_df['Forecast'])
rmse = np.sqrt(mean_squared_error(result_df['Actual'], result_df['Forecast']))
mape = np.mean(np.abs(result_df['Error (%)']))
r2 = r2_score(result_df['Actual'], result_df['Forecast'])

print("\n📈 Model Evaluation Metrics (Last 12 Months):")
print(f"MAE  : {mae:.2f}")
print(f"RMSE : {rmse:.2f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.3f}")

# ======== Residuals ========
residuals = result_df['Error']

# Residual plot
plt.figure(figsize=(10, 4))
plt.plot(residuals, marker='o', linestyle='-', color='purple')
plt.axhline(0, linestyle='--', color='gray')
plt.title("Residuals Plot (Forecast - Actual)")
plt.xlabel("Date")
plt.ylabel("Residual")
plt.grid(True)
plt.tight_layout()
plt.show()

# ACF of residuals
fig, ax = plt.subplots(figsize=(10, 4))
sm.graphics.tsa.plot_acf(residuals, lags=11, ax=ax)
plt.title("Autocorrelation of Residuals (ACF)")
plt.tight_layout()
plt.show()

# Histogram of residuals with KDE
plt.figure(figsize=(8, 4))
sns.histplot(residuals, bins=10, kde=True, color='teal')
plt.title("Distribution of Residuals (Histogram + KDE)")
plt.xlabel("Residual")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()



📊 1回目のデータ分割による予測解析の出力と見方

本システムでは、モデルの予測性能を客観的に評価するために、時系列データを以下の2つに分割して処理を行います：
* 学習データ（Training Data）：モデル構築（学習）に使用される過去のデータ
*	検証データ（Test Data）：構築されたモデルで予測し、実測値と比較するためのデータ

この段階で出力される結果は、モデルの適合性や予測性能を確認するための非常に重要な情報です。

⸻

🔢 出力される主な情報とその見方

1. 📈 実測値と予測値のグラフ
*	青い線：実測値
*	オレンジの線：予測値
*	これらを比較することで、モデルが実際の傾向をどの程度うまく捉えているかが一目で確認できます。

✅ 見るべきポイント：予測線が実測線と近いか？大きく外れている月があるか？

⸻

2. 📉 残差分析（予測誤差）
*	残差ヒストグラム：予測誤差（残差）の分布
*	左右対称で中心が0に近いなら、予測は偏りが少ないと考えられます。
*	Q-Qプロット：残差が正規分布に従うかを確認
*	点が直線上に並ぶほど、残差が正規的で信頼性の高い予測とみなせます。
*	自己相関プロット（ACF）：残差に周期性や構造が残っていないかの確認
*	自己相関がほぼ0なら、モデルがパターンを十分捉えていることを示します。

⸻

3. 📊 モデル評価指標（テストデータに基づく定量評価）
* MAE（平均絶対誤差）:誤差の絶対値の平均(値が小さいほど良好)
* RMSE（二乗平均平方根誤差）:大きな誤差に敏感に変動
(MAEと差が大きいと大きな誤差が混ざっている可能性あり)
* MAPE（平均絶対パーセント誤差）:誤差の割合
(一般的に 20% 以下で良好)
* R²（決定係数）:モデルの説明力
(1に近いほど良好)


📌 この分析の目的

この1回目の分割・予測は、将来予測に進む前に「モデルが十分に実用レベルであるか」を検証する工程です。
ここで得られた知見に基づいて、ユーザーがモデルの精度を確認する必要があります。

上のセルでモデルの精度が十分だと確認できたら本解析に進みます。
本解析では、入力されたデータの最終月から12ヶ月後までの予測を行います。

In [ ]:
# === 再学習：全データでモデルを再構築 ===
print("\n🔁 Performing Final Forecast Using All Available Data...")

# モデル再構築（全データで）
final_model = pm.auto_arima(
    ts,
    seasonal=True,
    m=12,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True
)

print("\n✅ モデル概要:")
print(final_model.summary())

# === 12ヶ月先まで予測（CSVの最終月から） ===
n_forecast = 12
future_forecast, future_conf_int = final_model.predict(n_periods=n_forecast, return_conf_int=True)

# 予測のインデックス（日付）を生成（すべて月末）
last_date = ts.index[-1]
forecast_index_future = pd.date_range(start=last_date + pd.offsets.MonthEnd(1), periods=n_forecast, freq='M')

# 予測と信頼区間をSeriesに変換
forecast_future = pd.Series(future_forecast, index=forecast_index_future)
lower_future = pd.Series(future_conf_int[:, 0], index=forecast_index_future)
upper_future = pd.Series(future_conf_int[:, 1], index=forecast_index_future)

# === プロット：実データ + 予測 ===
plt.figure(figsize=(14, 6))
plt.plot(ts.index, ts, label='Observed', color='blue', marker='s')
plt.plot(forecast_index_future, forecast_future, label='Forecast', color='red', linestyle='--', marker='o')
plt.fill_between(forecast_index_future, lower_future, upper_future, color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title("Final Forecast (12 Months Ahead)")
plt.xlabel("Date (Month End)")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 最後の12ヶ月のインデックスと予測・信頼区間を抽出
forecast_index_last_12 = forecast_index_future[-12:]
forecast_last_12 = forecast_future[-12:]
lower_last_12 = lower_future[-12:]
upper_last_12 = upper_future[-12:]

# プロット
plt.figure(figsize=(10, 5))
plt.plot(forecast_index_last_12, forecast_last_12, label='Forecast (Last 12 Months)', color='red', linestyle='--', marker='o')
plt.fill_between(forecast_index_last_12, lower_last_12, upper_last_12, color='pink', alpha=0.3, label='95% Confidence Interval')
plt.title("Final Forecast: Last 12 Months Only")
plt.xlabel("Date (Month End)")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# === Optional: 予測結果の表示 ===
forecast_df = pd.DataFrame({
    'Forecast': forecast_future,
    'Lower CI': lower_future,
    'Upper CI': upper_future
})

# 結果を表示
print("\n📊 Final 12-Month Forecast:")
print(forecast_df.round(2))


## ライセンス

MIT License

Copyright (c) 2025 Kazuki Kimura

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

-------------------------------------------------------------------------------

📦 本ソフトウェア「EZ_ARIMA」は、以下のオープンソースライブラリに依存しています：

| ライブラリ名     | ライセンス種別 | 備考                           |
|------------------|----------------|--------------------------------|
| pandas           | BSD License    | データフレーム操作             |
| numpy            | BSD License    | 数値計算                       |
| matplotlib       | PSF License    | グラフ描画                     |
| seaborn          | BSD License    | 統計可視化                     |
| statsmodels      | BSD License    | 時系列解析・回帰分析など       |
| pmdarima         | MIT License    | ARIMA/SARIMA 自動モデリング    |

これらのライブラリは本ソフトウェアには同梱されておらず、pip等を通じてインストールされます。  
使用ライブラリの各ライセンスは、それぞれの公式サイトまたはGitHubリポジトリで確認可能です。

-------------------------------------------------------------------------------

🛡 著作権・ライセンス遵守に関する声明

- 上記ライブラリは、そのライセンスに従い合法的に利用されています。
- 本ソフトウェアには、他者のコードや技術記事からの参照・派生要素を含むことがありますが、いずれもライセンス条件を遵守した上で構成されています。
- 著作権侵害の意図は一切なく、必要に応じて謝辞・出典等の追記や修正を行います。

問題がある場合は GitHub の Issues よりご連絡ください。確認後、速やかに対応いたします。

このソフトウェアは個人で制作したものであり郡山市及び郡山市上下水道局は制作に関係しておりません。